In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Colab Notebooks/Resized All Images.zip') as zipObj:
  zipObj.extractall()

# Resizer

In [ ]:
pip install Augmentor

In [ ]:
import Augmentor as ag

In [ ]:
pipe = ag.Pipeline("/content/Resized All Images", output_directory="/content/output2")
pipe.crop_by_size(probability = 1, height=1424, width=1424)
pipe.resize(probability = 1, width=300, height=300)
pipe.rotate(probability=0.8, max_left_rotation = 10, max_right_rotation = 10)  

In [ ]:
pipe.

In [ ]:
pipe.sample(10000)

# With GPU Augmentation

In [ ]:
import pathlib
import tensorflow as tf

data_dir = pathlib.Path('/content/output2')
img_height = 300
img_width = 300

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height,img_width),
    batch_size=2
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height,img_width),
    batch_size=2
)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
import numpy as np

normalized_ds = train_ds.map(lambda x,y: (normalization_layer(x),y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import ResNet152V2, InceptionV3, EfficientNetB4, DenseNet201
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D

# Model 1

base_model = InceptionV3(
    input_shape = (300,300,3),
    weights = "imagenet",
    include_top = False
)

for layer in base_model.layers[:10]:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.4)(x)

predictions = Dense(6, activation='softmax')(x)
model1 = Model(inputs=base_model.inputs, outputs = predictions)

In [ ]:
# Model 2

base_model = DenseNet201(
    input_shape = (300,300,3),
    weights = "imagenet",
    include_top = False
)

for layer in base_model.layers[:10]:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(6, activation='softmax')(x)
model2 = Model(inputs=base_model.inputs, outputs = predictions)

In [ ]:
# Model 3

base_model = EfficientNetB4(
    input_shape = (300,300,3),
    weights = "imagenet",
    include_top = False
)

for layer in base_model.layers[:10]:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(6, activation='softmax')(x)
model3 = Model(inputs=base_model.inputs, outputs = predictions)

In [ ]:
# Model 4

base_model = ResNet152V2(
    input_shape = (300,300,3),
    weights = "imagenet",
    include_top = False
)

for layer in base_model.layers[:10]:
  layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(6, activation='softmax')(x)
model4 = Model(inputs=base_model.inputs, outputs = predictions)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_filepath = "/content/drive/MyDrive/model2 - {epoch:02d} - {val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath = model_filepath,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only = True,
    verbose = 1
)

In [ ]:
model1.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
model2.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
model3.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
model4.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
with tf.device("/device:GPU:0"):
  history1 = model1.fit(
      train_ds,
      validation_data = val_ds,
      epochs = 7,
      callbacks = [checkpoint]
  )

In [ ]:
with tf.device("/device:GPU:0"):
  history2 = model2.fit(
      train_ds,
      validation_data = val_ds,
      epochs = 7,
      callbacks = [checkpoint]
  )

In [ ]:
with tf.device("/device:GPU:0"):
  history3 = model3.fit(
      train_ds,
      validation_data = val_ds,
      epochs = 7,
      callbacks = [checkpoint]
  )

In [ ]:
with tf.device("/device:GPU:0"):
  history4 = model4.fit(
      train_ds,
      validation_data = val_ds,
      epochs = 7,
      callbacks = [checkpoint]
  )

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

model_1 = load_model(model_filepath)
model_2 = load_model(model_filepath)
model_3 = load_model(model_filepath)
model_4 = load_model(model_filepath)

models = [model_1, model_2, model_3, model_4]
model_input = Input(shape=(300,300,3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs = model_input, outputs = ensemble_output, name='ensemble')

In [ ]:
ensemble_model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossEntropy(from_logits=False), metrics=['accuracy'])

In [ ]:
history = ensemble_model.fit(
    train_ds,
    val_ds,
    epochs = 5,
    callbacks = [checkpoint]
)